# **Organización de Datos 75.06 - Trabajo Práctico N°2**
S13. Si decimos que un usuario es adyacente a otro cuando le dieron un rating a una misma película. ¿Cuántos usuarios son adyacentes al 179792? ¿Cuántos son adyacentes a sus adyacentes?

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

from google.colab import drive
drive.mount('/content/drive')

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

ratings = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/ratings.parquet').rdd

In [ ]:
ratingsManual = sc.parallelize([('toy_story', 2), ('toy_story', 4), ('toy_story', 6), ('toy_story', 179792), ('cars', 179792), ('monsters inc', 2), ('monsters inc', 8), ('monsters inc', 179792), ('harry potter', 4), ('harry potter', 7)])
#Los adyacentes son 2, 4, 6, 8.
#Los adyacentes de los adyacentes son 179792 y 7.

In [2]:
ratingsSample = sqlContext.read.parquet('/content/drive/MyDrive/Colab Notebooks/TP2/parquet/ratings.parquet').rdd.sample(False, 0.025)

In [3]:
#ratingsPeliUsuario = ratings.map(lambda x: (x.movieId, x.userId)) #Uso un sample porque el archivo original es muy pesado.

ratingsPeliUsuario = ratingsSample.map(lambda x: (x.movieId, x.userId)).cache()

ratings_179792 = ratingsPeliUsuario.filter(lambda x: x[1] == 179792)
ratings_sin_179792 = ratingsPeliUsuario.filter(lambda x: x[1] != 179792)

ratings_adyacentes = ratings_179792.join(ratings_sin_179792).map(lambda x: (x[1][1], x[0])).cache() #Ratings de los adyacentes como, mapeado como (usuario, pelicula)
adyacentes = ratings_adyacentes.map(lambda x: x[0]).distinct()

print("Hay", adyacentes.count(), "usuarios adyacentes al usuario 179792.")

Hay 10224 usuarios adyacentes al usuario 179792.


In [4]:
ratingsUsuarioPeli = ratingsPeliUsuario.map(lambda x: (x[1], x[0])) #Ratings como (usuario, pelicula)
ratings_totales_adyacentes = ratingsUsuarioPeli.join(ratings_adyacentes).map(lambda x: (x[1][0], x[0])) #Todos los ratings hechos por los usuarios adyacentes, como (pelicula, usuario)

ratings_pelis_votadas_por_adyacentes = ratingsPeliUsuario.join(ratings_totales_adyacentes).map(lambda x: (x[1][0], x[0])).distinct() #Ratings de peliculas que votaron los adyacentes.
#Los adyacentes de los adyacentes serán los usuarios de ratings_pelis_votadas_por_adyacentes con id DISTINTO al de los adyacentes.

adyacentes_adyacentes = ratings_pelis_votadas_por_adyacentes.subtractByKey(ratings_adyacentes).map(lambda x: x[0]).distinct()

print("Hay", adyacentes_adyacentes.count(), "usuarios adyacentes a los usuarios adyacentes.")

Hay 143898 usuarios adyacentes a los usuarios adyacentes.
